In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import FloatType
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType


off_train = spark.read.csv('ccf_offline_stage1_train.csv', header=True, inferSchema=True)
off_test = spark.read.csv('ccf_offline_stage1_test_revised.csv', header = True, inferSchema=True)
on_train = spark.read.csv('ccf_online_stage1_train.csv', header = True, inferSchema = True)

In [2]:
dataset3 = off_test
feature3 = off_train.filter(((off_train.Date>='20160315')&(off_train.Date<='20160630'))|((off_train.Date=='null') & (off_train.Date_received>='20160315')&(off_train.Date_received<='20160630'))).select('*')
dataset2 = off_train.filter((off_train.Date_received>='20160515')&(off_train.Date_received<='20160615')).select('*')
feature2 = off_train.filter((off_train.Date>='20160201')&(off_train.Date<='20160514')|((off_train.Date=='null')&(off_train.Date_received>='20160201')&(off_train.Date_received<='20160514'))).select('*')
dataset1 = off_train.filter((off_train.Date_received>='20160414')&(off_train.Date_received<='20160514')).select('*')
feature1 = off_train.filter((off_train.Date>='20160101')&(off_train.Date<='20160413')|((off_train.Date=='null')&(off_train.Date_received>='20160101')&(off_train.Date_received<='20160413'))).select('*')

In [3]:
#dataset3.show()
#dataset2.show()
#dataset1.show()
#feature3.show()

+-------+-----------+---------+-------------+--------+-------------+
|User_id|Merchant_id|Coupon_id|Discount_rate|Distance|Date_received|
+-------+-----------+---------+-------------+--------+-------------+
|4129537|        450|     9983|         30:5|       1|     20160712|
|6949378|       1300|     3429|         30:5|    null|     20160706|
|2166529|       7113|     6928|       200:20|       5|     20160727|
|2166529|       7113|     1808|       100:10|       5|     20160727|
|6172162|       7605|     6500|         30:1|       2|     20160708|
|4005121|        450|     9983|         30:5|       0|     20160706|
|4347394|        450|     9983|         30:5|       0|     20160716|
|3094273|        760|    13602|         30:5|       1|     20160727|
|5139970|        450|     9983|         30:5|      10|     20160729|
|3237121|        760|    13602|         30:5|       1|     20160703|
|6224386|        450|     9983|         30:5|       3|     20160716|
|6488578|        760|    13602|   

In [3]:
###5. other feature:###

#for dataset3
t = dataset3.select('User_id')
t = t.select('User_id').groupBy('User_id').count().withColumnRenamed('count', 'Month_Receive_all_coupon_count')

#print(t.orderBy('User_id').show())

In [4]:
t1 = dataset3.select('User_id','Coupon_id')
t1 = t1.select('User_id','Coupon_id').groupBy('User_id','Coupon_id').count().withColumnRenamed('Count', 'Month_Receive_same_coupon_count')

#print(t1.orderBy('User_id').show())

In [5]:
t2 = dataset3.select('User_id', 'Coupon_id', 'Date_received')

In [6]:
from pyspark.sql import functions as F
t2 = t2.groupby(['User_id', 'Coupon_id']).agg(F.max("Date_received"),F.min("Date_received"))
t2 = t2.withColumnRenamed('max(Date_received)','Max_date_received').withColumnRenamed('Min(Date_received)','Min_date_received')
t2.show()

+-------+---------+-----------------+-----------------+
|User_id|Coupon_id|Max_date_received|Min_date_received|
+-------+---------+-----------------+-----------------+
|3649795|    10418|         20160701|         20160701|
|3025159|    10438|         20160701|         20160701|
|2296075|    13602|         20160728|         20160728|
|7346734|     4405|         20160701|         20160701|
|2970925|     5178|         20160728|         20160711|
|2746177|    13602|         20160704|         20160704|
|5632654|     4138|         20160716|         20160716|
|5406394|    13602|         20160705|         20160705|
|4451026|     9983|         20160711|         20160711|
|2891131|     3429|         20160724|         20160724|
|3032473|     8306|         20160715|         20160715|
|2447773|    12807|         20160703|         20160703|
|6210370|     2978|         20160704|         20160704|
|4541266|     9983|         20160719|         20160719|
|3618223|    11799|         20160712|         20

In [7]:
t3 = dataset3.select('User_id','Coupon_id','Date_received')
t3 = t3.join(t2, ['User_id','Coupon_id'])
t3 = t3.withColumn('Month_same_coupon_lastone', col('Max_date_received') - col('date_received'))
t3 = t3.withColumn('Month_same_coupon_firstone',col('date_received') - col('Min_date_received'))                   

#print(t3.orderBy('User_id').show())

In [8]:
equally1 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_lastone', lit(1).alias('feat1')).filter('Month_same_coupon_lastone == 0').drop('Month_same_coupon_lastone')
greater1 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_lastone', lit(0).alias('feat1')).filter('Month_same_coupon_lastone > 0').drop('Month_same_coupon_lastone')
u1 = equally1.union(greater1).withColumnRenamed('feat1', 'Month_same_coupon_lastone')
u1 = u1.select('User_id','Coupon_id','Date_received','Month_same_coupon_lastone')
t3 = t3.drop('Month_same_coupon_lastone')

equally2 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_firstone', lit(1).alias('feat2')).filter('Month_same_coupon_firstone == 0').drop('Month_same_coupon_firstone')
greater2 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_firstone', lit(0).alias('feat2')).filter('Month_same_coupon_firstone > 0').drop('Month_same_coupon_firstone')
u2 = equally2.union(greater2).withColumnRenamed('feat2', 'Month_same_coupon_firstone')
u2 = u2.select('User_id','Coupon_id','Date_received','Month_same_coupon_firstone')
t3 = t3.drop('Month_same_coupon_firstone')

t3 = t3.join(u1, ['User_id','Coupon_id','Date_received'])
t3 = t3.join(u2, ['User_id','Coupon_id','Date_received'])
t3 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_lastone','Month_same_coupon_firstone')

#print(t3.orderBy('User_id').show())
#print(u1.orderBy('User_id').show())
#print(u2.orderBy('User_id').show())

In [9]:
t4 = dataset3.select('User_id','Date_received')
t4 = t4.select('Date_received','User_id',).groupBy('User_id','Date_received').count().withColumnRenamed('count', 'Day_Receive_all_coupon_count')

#print(t4.orderBy('user_id').show())

+-------+-------------+----------------------------+
|User_id|Date_received|Day_Receive_all_coupon_count|
+-------+-------------+----------------------------+
|    209|     20160721|                           2|
|    215|     20160703|                           1|
|    316|     20160721|                           1|
|    417|     20160712|                           1|
|    432|     20160706|                           1|
|    448|     20160701|                           1|
|    448|     20160707|                           1|
|    448|     20160704|                           1|
|    448|     20160710|                           1|
|    448|     20160706|                           1|
|    452|     20160702|                           1|
|    452|     20160721|                           1|
|    470|     20160710|                           1|
|    569|     20160716|                           1|
|    728|     20160714|                           1|
|    736|     20160724|                       

In [10]:
t5 = dataset3.select('User_id','Coupon_id','Date_received')
t5 = t5.select('User_id','Coupon_id','Date_received').groupBy('User_id','Coupon_id','Date_received').count().withColumnRenamed('count', 'Day_Receive_same_coupon_count')

#print(t5.orderBy('user_id').show())

+-------+---------+-------------+-----------------------------+
|User_id|Coupon_id|Date_received|Day_Receive_same_coupon_count|
+-------+---------+-------------+-----------------------------+
|    209|      825|     20160721|                            1|
|    209|     7557|     20160721|                            1|
|    215|     5488|     20160703|                            1|
|    316|     3992|     20160721|                            1|
|    417|    12465|     20160712|                            1|
|    432|    10438|     20160706|                            1|
|    448|    10927|     20160706|                            1|
|    448|    13602|     20160701|                            1|
|    448|    10927|     20160710|                            1|
|    448|    10927|     20160704|                            1|
|    448|    10927|     20160707|                            1|
|    452|     3992|     20160721|                            1|
|    452|     6500|     20160702|       

In [11]:
#商家发出的所有优惠券种类数目  label_merchant_coupon_type_count

t6 = dataset3.select('Merchant_id','Coupon_id')
t6 = t6.select('Merchant_id','Coupon_id').groupBy('Merchant_id','Coupon_id').count().withColumnRenamed('count', 'Label_merchant_coupon_count')


#print(t6.orderBy(desc('Label_merchant_coupon_count')).show())
#print(t6.orderBy('Merchant_id').show())

+-----------+---------+---------------------------+
|Merchant_id|Coupon_id|Label_merchant_coupon_count|
+-----------+---------+---------------------------+
|        760|    13602|                      29885|
|        450|     9983|                      11586|
|       5717|    10418|                       7370|
|       5138|     8059|                       6401|
|       3621|     2978|                       5339|
|       1300|     3429|                       4887|
|       2050|    10438|                       4215|
|       1469|    11799|                       1725|
|       2436|     3992|                       1650|
|       7717|     1904|                       1425|
|       1469|     2601|                       1019|
|       7717|    11894|                        797|
|       6901|      785|                        606|
|       7354|     2002|                        557|
|       7422|     4727|                        533|
|       3939|     4277|                        492|
|       4582

In [12]:
#用户领取该商家的所有优惠券数目  label_user_merchant_coupon_count

t7 = dataset3.select('user_id','Merchant_id')
t7 = t7.select('user_id','Merchant_id').groupBy('user_id','Merchant_id').count().withColumnRenamed('count', 'Label_user_merchant_coupon_count')

#print(t7.orderBy('user_id').show())

In [13]:
#用户在此次优惠券之后还领取了多少相同优惠券     label_coupon_count_later


from pyspark.sql.window import Window
from pyspark.sql import functions as F

t8 = dataset3.select('User_id','Coupon_id','Date_received')
win1 = Window.partitionBy('User_id','Coupon_id').orderBy(desc('Date_received'))
t8= t8.withColumn("rank",F.rank().over(win1))
t8 = t8.select('User_id','Coupon_id','Date_received', lit(t8['rank']-1).alias('Coupon_count_later'))


#print(t8.orderBy('user_id').show())

In [14]:
other_feature3 = t1.join(t, 'User_id')
other_feature3 = other_feature3.join(t3,['user_id','coupon_id'])
other_feature3 = other_feature3.join(t4, ['User_id','Date_received'])
other_feature3 = other_feature3.join(t5, ['User_id','Coupon_id','Date_received'])
other_feature3 = other_feature3.join(t6, ['Coupon_id'])
other_feature3 = other_feature3.join(t7, ['user_id','Merchant_id'])
other_feature3 = other_feature3.join(t8, ['User_id','Coupon_id','Date_received'])


#other_feature3.write.csv('other_feature3', header = True)

#other_feature3.show()

In [15]:
other_feature3.coalesce(1).write.mode("overwrite").option('header','true').csv('other_feature3')

In [ ]:
#for dataset2

In [3]:
t = dataset2.select('User_id')
t = t.select('User_id').groupBy('User_id').count().withColumnRenamed('count', 'Month_Receive_all_coupon_count')

#print(t.orderBy('User_id').show())

t1 = dataset2.select('User_id','Coupon_id')
t1 = t1.select('User_id','Coupon_id').groupBy('User_id','Coupon_id').count().withColumnRenamed('Count', 'Month_Receive_same_coupon_count')

#print(t1.orderBy('User_id').show())

In [4]:
t2 = dataset2.select('User_id', 'Coupon_id', 'Date_received')

from pyspark.sql import functions as F
t2 = t2.groupby(['User_id', 'Coupon_id']).agg(F.max("Date_received"),F.min("Date_received"))
t2 = t2.withColumnRenamed('max(Date_received)','Max_date_received').withColumnRenamed('Min(Date_received)','Min_date_received')
#t2.show()

In [5]:
t3 = dataset2.select('User_id','Coupon_id','Date_received')
t3 = t3.join(t2, ['User_id','Coupon_id'])
t3 = t3.withColumn('Month_same_coupon_lastone', col('Max_date_received') - col('date_received'))
t3 = t3.withColumn('Month_same_coupon_firstone',col('date_received') - col('Min_date_received'))                   

#print(t3.orderBy('User_id').show())


equally1 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_lastone', lit(1).alias('feat1')).filter('Month_same_coupon_lastone == 0').drop('Month_same_coupon_lastone')
greater1 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_lastone', lit(0).alias('feat1')).filter('Month_same_coupon_lastone > 0').drop('Month_same_coupon_lastone')
u1 = equally1.union(greater1).withColumnRenamed('feat1', 'Month_same_coupon_lastone')
u1 = u1.select('User_id','Coupon_id','Date_received','Month_same_coupon_lastone')
t3 = t3.drop('Month_same_coupon_lastone')

equally2 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_firstone', lit(1).alias('feat2')).filter('Month_same_coupon_firstone == 0').drop('Month_same_coupon_firstone')
greater2 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_firstone', lit(0).alias('feat2')).filter('Month_same_coupon_firstone > 0').drop('Month_same_coupon_firstone')
u2 = equally2.union(greater2).withColumnRenamed('feat2', 'Month_same_coupon_firstone')
u2 = u2.select('User_id','Coupon_id','Date_received','Month_same_coupon_firstone')
t3 = t3.drop('Month_same_coupon_firstone')

t3 = t3.join(u1, ['User_id','Coupon_id','Date_received'])
t3 = t3.join(u2, ['User_id','Coupon_id','Date_received'])
t3 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_lastone','Month_same_coupon_firstone')

#print(t3.orderBy('User_id').show())
#print(u1.orderBy('User_id').show())
#print(u2.orderBy('User_id').show())

In [6]:
t4 = dataset2.select('User_id','Date_received')
t4 = t4.select('Date_received','User_id',).groupBy('User_id','Date_received').count().withColumnRenamed('count', 'Day_Receive_all_coupon_count')

#print(t4.orderBy('user_id').show())


t5 = dataset2.select('User_id','Coupon_id','Date_received')
t5 = t5.select('User_id','Coupon_id','Date_received').groupBy('User_id','Coupon_id','Date_received').count().withColumnRenamed('count', 'Day_Receive_same_coupon_count')

#print(t5.orderBy('user_id').show())


In [7]:
#商家发出的所有优惠券种类数目  label_merchant_coupon_type_count

t6 = dataset2.select('Merchant_id','Coupon_id')
t6 = t6.select('Merchant_id','Coupon_id').groupBy('Merchant_id','Coupon_id').count().withColumnRenamed('count', 'Label_merchant_coupon_count')


#print(t6.orderBy(desc('Label_merchant_coupon_count')).show())
#print(t6.orderBy('Merchant_id').show())


#用户领取该商家的所有优惠券数目  label_user_merchant_coupon_count

t7 = dataset2.select('user_id','Merchant_id')
t7 = t7.select('user_id','Merchant_id').groupBy('user_id','Merchant_id').count().withColumnRenamed('count', 'Label_user_merchant_coupon_count')

#print(t7.orderBy('user_id').show())

#用户在此次优惠券之后还领取了多少相同优惠券     label_coupon_count_later


from pyspark.sql.window import Window
from pyspark.sql import functions as F

t8 = dataset2.select('User_id','Coupon_id','Date_received')
win1 = Window.partitionBy('User_id','Coupon_id').orderBy(desc('Date_received'))
t8= t8.withColumn("rank",F.rank().over(win1))
t8 = t8.select('User_id','Coupon_id','Date_received', lit(t8['rank']-1).alias('Coupon_count_later'))


#print(t8.orderBy('user_id').show())

In [9]:
other_feature2 = t1.join(t, 'User_id')
other_feature2 = other_feature2.join(t3,['user_id','coupon_id'])
other_feature2 = other_feature2.join(t4, ['User_id','Date_received'])
other_feature2 = other_feature2.join(t5, ['User_id','Coupon_id','Date_received'])
other_feature2 = other_feature2.join(t6, ['Coupon_id'])
other_feature2 = other_feature2.join(t7, ['user_id','Merchant_id'])
other_feature2 = other_feature2.join(t8, ['User_id','Coupon_id','Date_received'])


#other_feature2.write.csv('other_feature2', header = True)

#other_feature2.show()

In [9]:
#other_feature2 = t7.join(t6, ['Merchant_id'])
#other_feature2 = other_feature2.join(t8, ['User_id','Coupon_id'])

#other_feature2.show()

In [10]:
other_feature2.coalesce(1).write.mode("overwrite").option('header','true').csv('other_feature2')

In [1]:
#for dataset1

In [17]:
t = dataset1.select('User_id')
t = t.select('User_id').groupBy('User_id').count().withColumnRenamed('count', 'Month_Receive_all_coupon_count')

#print(t.orderBy('User_id').show())

In [18]:
t1 = dataset1.select('User_id','Coupon_id')
t1 = t1.select('User_id','Coupon_id').groupBy('User_id','Coupon_id').count().withColumnRenamed('Count', 'Month_Receive_same_coupon_count')

#print(t1.orderBy('User_id').show())

In [19]:
t2 = dataset1.select('User_id', 'Coupon_id', 'Date_received')

In [20]:
from pyspark.sql import functions as F
t2 = t2.groupby(['User_id', 'Coupon_id']).agg(F.max("Date_received"),F.min("Date_received"))
t2 = t2.withColumnRenamed('max(Date_received)','Max_date_received').withColumnRenamed('Min(Date_received)','Min_date_received')
#t2.show()

In [21]:
t3 = dataset1.select('User_id','Coupon_id','Date_received')
t3 = t3.join(t2, ['User_id','Coupon_id'])
t3 = t3.withColumn('Month_same_coupon_lastone', col('Max_date_received') - col('date_received'))
t3 = t3.withColumn('Month_same_coupon_firstone',col('date_received') - col('Min_date_received'))                   
#t3.show()

In [22]:
equally1 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_lastone', lit(1).alias('feat1')).filter('Month_same_coupon_lastone == 0').drop('Month_same_coupon_lastone')
greater1 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_lastone', lit(0).alias('feat1')).filter('Month_same_coupon_lastone > 0').drop('Month_same_coupon_lastone')
u1 = equally1.union(greater1).withColumnRenamed('feat1', 'Month_same_coupon_lastone')
u1 = u1.select('User_id','Coupon_id','Date_received','Month_same_coupon_lastone')
t3 = t3.drop('Month_same_coupon_lastone')

equally2 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_firstone', lit(1).alias('feat2')).filter('Month_same_coupon_firstone == 0').drop('Month_same_coupon_firstone')
greater2 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_firstone', lit(0).alias('feat2')).filter('Month_same_coupon_firstone > 0').drop('Month_same_coupon_firstone')
u2 = equally2.union(greater2).withColumnRenamed('feat2', 'Month_same_coupon_firstone')
u2 = u2.select('User_id','Coupon_id','Date_received','Month_same_coupon_firstone')
t3 = t3.drop('Month_same_coupon_firstone')

t3 = t3.join(u1, ['User_id','Coupon_id','Date_received'])
t3 = t3.join(u2, ['User_id','Coupon_id','Date_received'])
t3 = t3.select('User_id','Coupon_id','Date_received','Month_same_coupon_lastone','Month_same_coupon_firstone')

#print(t3.orderBy('User_id').show())
#print(u1.orderBy('User_id').show())
#print(u2.orderBy('User_id').show())

In [23]:
t4 = dataset1.select('User_id','Date_received')
t4 = t4.select('Date_received','User_id',).groupBy('User_id','Date_received').count().withColumnRenamed('count', 'Day_Receive_all_coupon_count')

#print(t4.orderBy('user_id').show())

In [24]:
t5 = dataset1.select('User_id','Coupon_id','Date_received')
t5 = t5.select('User_id','Coupon_id','Date_received').groupBy('User_id','Coupon_id','Date_received').count().withColumnRenamed('count', 'Day_Receive_same_coupon_count')

#print(t5.orderBy('user_id').show())

In [3]:
#商家发出的所有优惠券种类数目  label_merchant_coupon_type_count

t6 = dataset1.select('Merchant_id','Coupon_id')
t6 = t6.select('Merchant_id','Coupon_id').groupBy('Merchant_id','Coupon_id').count().withColumnRenamed('count', 'Label_merchant_coupon_count')


#print(t6.orderBy(desc('Label_merchant_coupon_count')).show())
#print(t6.orderBy('Merchant_id').show())

In [4]:
#用户领取该商家的所有优惠券数目  label_user_merchant_coupon_count

t7 = dataset1.select('user_id','Merchant_id')
t7 = t7.select('user_id','Merchant_id').groupBy('user_id','Merchant_id').count().withColumnRenamed('count', 'Label_user_merchant_coupon_count')

print(t7.orderBy('user_id').show())

+-------+-----------+--------------------------------+
|user_id|Merchant_id|Label_user_merchant_coupon_count|
+-------+-----------+--------------------------------+
|    173|       3381|                               1|
|    285|        450|                               1|
|    316|       7974|                               1|
|    377|       4906|                               1|
|    387|       3381|                               1|
|    430|       7555|                               1|
|    467|       7555|                               1|
|    470|       7717|                               1|
|    472|       2436|                               1|
|    489|       6568|                               1|
|    489|       4057|                               1|
|    597|       1569|                               1|
|    798|        797|                               1|
|    865|       7993|                               1|
|    892|       1569|                               1|
|    947| 

In [5]:
#用户在此次优惠券之后还领取了多少相同优惠券     label_coupon_count_later


from pyspark.sql.window import Window
from pyspark.sql import functions as F

t8 = dataset1.select('User_id','Coupon_id','Date_received')
win1 = Window.partitionBy('User_id','Coupon_id').orderBy(desc('Date_received'))
t8= t8.withColumn("rank",F.rank().over(win1))
t8 = t8.select('User_id','Coupon_id','Date_received', lit(t8['rank']-1).alias('Coupon_count_later'))


#print(t8.orderBy('user_id').show())

In [28]:
other_feature1 = t1.join(t, 'User_id')
other_feature1 = other_feature1.join(t3,['user_id','coupon_id'])
other_feature1 = other_feature1.join(t4, ['User_id','Date_received'])
other_feature1 = other_feature1.join(t5, ['User_id','Coupon_id','Date_received'])
other_feature1 = other_feature1.join(t6, ['Coupon_id'])
other_feature1 = other_feature1.join(t7, ['user_id','Merchant_id'])
other_feature1 = other_feature1.join(t8, ['User_id','Coupon_id','Date_received'])


#other_feature2.write.csv('other_feature2', header = True)

#other_feature2.show()

In [7]:
#other_feature1 = t7.join(t6, ['Merchant_id'])
#other_feature1 = other_feature1.join(t8, ['User_id','Coupon_id'])

In [8]:
other_feature1.coalesce(1).write.mode("overwrite").option('header','true').csv('other_feature1')